# Managing Available Tools for e-Commerce Customer Service

## Overview

This notebook focuses on identifying and managing the **tools and APIs** required for an AI-powered e-commerce customer service assistant. These tools enable the assistant to execute structured actions based on the **e-Commerce Customer Service Policy**, ensuring efficiency, compliance, and accuracy in handling customer inquiries.

## Purpose of Tool Management

By defining the available tools and their capabilities, we establish a framework that allows the assistant to:
- **Retrieve order-related information** (e.g., order status, shipment tracking, order history).
- **Manage customer requests** (e.g., modify orders, initiate returns, process refunds).
- **Ensure seamless customer account operations** (e.g., account updates, password resets, approved user verification).
- **Handle complaints and escalation processes** effectively.

Each tool corresponds to a function, with specific parameters that dictate how requests are processed.

<div align="center">
  <img src="images/tools.png" alt="Tools" width="800">
</div>

## Available Functions and Parameters

| **Function Name**            | **Description**                                              | **Required Parameters**                   |
|------------------------------|--------------------------------------------------------------|-------------------------------------------|
| `get_order_status`           | Retrieve the current status of a customer order.             | `order_id`                                |
| `modify_order`               | Adjust an order (e.g., change shipping address, quantity).   | `order_id`, `modification_details`        |
| `cancel_order`               | Cancel an order before shipment.                             | `order_id`                                |
| `track_shipment`             | Fetch tracking details for a shipped order.                  | `order_id`                                |
| `initiate_return`            | Process a return request for an order.                       | `order_id`, `reason`                      |
| `check_refund_eligibility`   | Verify if an order qualifies for a refund.                   | `order_id`                                |
| `process_refund`             | Issue a refund to a customer.                                | `order_id`, `refund_amount`               |
| `get_customer_order_history` | Retrieve a customer’s past orders.                           | `customer_id`                             |
| `handle_complaint`           | Address and resolve customer disputes.                       | `complaint_details`                       |
| `escalate_to_human`          | Escalate an issue to a human representative.                 | `issue_details`                           |
| `update_customer_account`    | Modify customer account details (e.g., email, password).     | `customer_id`, `update_type`, `new_value` |
| `get_faq_response`           | Retrieve automated responses for frequently asked questions. | `faq_topic`                               |

## Getting Started

Prerequisites:

- Python 3.11 or 3.12 installed on your system.
- LLM Agentic Tool Mesh library installed. If not, install it using: `pip install 'llmesh[chat, agents]'`
- Install kernel for Jupiter notebook `pip install ipykernel`
- Create `.env` file inside `notebooks/data` folder with the value of `PYTHONPATH` environment variables

In [1]:
from dotenv import load_dotenv
import os

# Load environment variables from the .env file located in the Data folder
dotenv_path = os.path.join('data', '.env')
load_dotenv(dotenv_path)

# Read environment variables
python_path = os.getenv('PYTHONPATH')
if not python_path:
    raise ValueError("Project path is not set.")

# Change working directory
os.chdir(python_path)

In [ ]:
import json
from IPython.display import Markdown
from notebooks.meta_prompting.utils import load_prompt, TOOLS, TOOL_CLOSE

tools = f"\n\n```json\n{json.dumps(TOOLS, indent=2)}\n```"
system_prompt = load_prompt('routine_generation', {'tools':tools, 'tool_close': TOOL_CLOSE})
display(Markdown(f"## **E-Commerce Customer Service Meta Prompt with Tools**\n\n{system_prompt}"))

/Users/antoniofin/Documents/Code/llmesh/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


2025-03-09 23:03:22,556 - ATHON - DEBUG - Prompt generated from notebooks/meta_prompting/prompts//routine_generation.txt with params {'tools': '\n\n```json\n[\n  {\n    "name": "get_order_status",\n    "description": "Fetch the current status of an order given its order ID. Possible statuses: Processing, Shipped, Delivered, Cancelled.",\n    "parameters": {\n      "order_id": {\n        "description": "The unique identifier of the order to retrieve its status.",\n        "type": "string",\n        "required": true\n      }\n    }\n  },\n  {\n    "name": "modify_order",\n    "description": "Modify an existing order, such as updating the shipping address or item quantity.",\n    "parameters": {\n      "order_id": {\n        "description": "The order ID to modify.",\n        "type": "string",\n        "required": true\n      },\n      "modification_details": {\n        "description": "Details of the requested modification.",\n        "type": "string",\n        "required": true\n      }\n 

## **E-Commerce Customer Service Meta Prompt with Tools**

You are a helpful assistant tasked with taking an external facing operational guide and converting it into a internal-facing programmatically executable routine optimized for an LLM. 
The LLM using this routine will be tasked with reading the policy, answering incoming questions from customers, and helping drive the case toward resolution.

Please follow these instructions:
1. **Review the operational guide policy carefully** to ensure every step is accounted for. It is crucial not to skip any steps or policies.
2. **Organize the instructions into a logical, step-by-step order**, using the specified format. 
3. **Use the following format**:
   - **Main actions are numbered** (e.g., 1, 2, 3).
   - **Sub-actions are lettered** under their relevant main actions (e.g., 1a, 1b).
      **Sub-actions should start on new lines**
   - **Specify conditions using clear 'if...then...else' statements** (e.g., 'If the product was purchased within 30 days, then...').
   - **For instructions that require more information from the customer**, provide polite and professional prompts to ask for additional information.
   - **For actions that require data from external systems**, write a step to call a function using backticks for the function name (e.g., call the `check_delivery_date` function).
      - **If a step requires the LLM agent to take an action** (e.g., check a metric), generate a function call for this action (e.g., call the `get_metric` function).
      - **Only use the available set of functions that are defined below.
   - **If there is an action an assistant can perform on behalf of the user**, include a function call for this action (e.g., call the `change_email_address` function), and ensure the function is defined with its purpose and required parameters.
      - **Only use the available set of functions that are defined below.
   - **The step prior to case resolution should always be to ask if there is anything more you can assist with**.
   - **End with a final action for case resolution**: calling the `close_case` function should always be the final step.
4. **Ensure compliance** by making sure all steps adhere to company policies, privacy regulations, and legal requirements.
5. **Handle exceptions or escalations** by specifying steps for scenarios that fall outside the standard policy.
6. **Ensure coverage** by checking that all of the conditions covered in the policy are also covered in the routines

**Important**: Always wrap the functions you return in backticks i.e. `check_ticket_type`. Do not include the arguments to the functions.

Here are the currently available set of functions in JSON format: 
TOOLS: 

```json
[
  {
    "name": "get_order_status",
    "description": "Fetch the current status of an order given its order ID. Possible statuses: Processing, Shipped, Delivered, Cancelled.",
    "parameters": {
      "order_id": {
        "description": "The unique identifier of the order to retrieve its status.",
        "type": "string",
        "required": true
      }
    }
  },
  {
    "name": "modify_order",
    "description": "Modify an existing order, such as updating the shipping address or item quantity.",
    "parameters": {
      "order_id": {
        "description": "The order ID to modify.",
        "type": "string",
        "required": true
      },
      "modification_details": {
        "description": "Details of the requested modification.",
        "type": "string",
        "required": true
      }
    }
  },
  {
    "name": "cancel_order",
    "description": "Cancel an order before it has been shipped.",
    "parameters": {
      "order_id": {
        "description": "The order ID to cancel.",
        "type": "string",
        "required": true
      }
    }
  },
  {
    "name": "track_shipment",
    "description": "Fetch tracking details for a shipped order.",
    "parameters": {
      "order_id": {
        "description": "The order ID for tracking information.",
        "type": "string",
        "required": true
      }
    }
  },
  {
    "name": "initiate_return",
    "description": "Process a return request for an order.",
    "parameters": {
      "order_id": {
        "description": "The order ID for return request.",
        "type": "string",
        "required": true
      },
      "reason": {
        "description": "Reason for returning the item.",
        "type": "string",
        "required": true
      }
    }
  },
  {
    "name": "check_refund_eligibility",
    "description": "Verify if an order qualifies for a refund.",
    "parameters": {
      "order_id": {
        "description": "The order ID to check refund eligibility.",
        "type": "string",
        "required": true
      }
    }
  },
  {
    "name": "process_refund",
    "description": "Process a refund request for an order.",
    "parameters": {
      "order_id": {
        "description": "The order ID for the refund request.",
        "type": "string",
        "required": true
      },
      "refund_amount": {
        "description": "The amount to be refunded.",
        "type": "string",
        "required": true
      }
    }
  },
  {
    "name": "get_customer_order_history",
    "description": "Retrieve a customer\u2019s past orders.",
    "parameters": {
      "customer_id": {
        "description": "The unique identifier of the customer.",
        "type": "string",
        "required": true
      }
    }
  },
  {
    "name": "handle_complaint",
    "description": "Log and address customer complaints or disputes.",
    "parameters": {
      "complaint_details": {
        "description": "Details of the customer complaint.",
        "type": "string",
        "required": true
      }
    }
  },
  {
    "name": "escalate_to_human",
    "description": "Escalate an issue to a human representative for further assistance.",
    "parameters": {
      "issue_details": {
        "description": "Details of the issue requiring human intervention.",
        "type": "string",
        "required": true
      }
    }
  },
  {
    "name": "update_customer_account",
    "description": "Modify customer account details such as email, password, or phone number.",
    "parameters": {
      "customer_id": {
        "description": "The customer ID whose account is to be updated.",
        "type": "string",
        "required": true
      },
      "update_type": {
        "description": "Type of update (e.g., email, password, phone number).",
        "type": "string",
        "required": true
      },
      "new_value": {
        "description": "New value for the update field.",
        "type": "string",
        "required": true
      }
    }
  },
  {
    "name": "get_faq_response",
    "description": "Retrieve automated responses for frequently asked questions (e.g., returns, refunds, shipping).",
    "parameters": {
      "faq_topic": {
        "description": "The FAQ topic to retrieve information about.",
        "type": "string",
        "required": true
      }
    }
  },
  {
    "name": "close_case",
    "description": "Tool to close a case.",
    "parameters": {
      "order_id": {
        "description": "The ID of the order",
        "type": "string",
        "required": true
      }
    }
  }
]
```

Please convert the following operational guide policy into the formatted routine, ensuring it is easy to follow and execute programmatically. Ensure that you **only** use the functions provided and do **not** create net new functions.

## Next Steps

In the next notebook ([**04: Generating the Meta-Prompt**](04_Generating_Meta_Prompt.ipynb)), we will structure the assistant’s system prompt to ensure it utilizes these tools effectively in real-world interactions.